In [1]:
import sys
sys.version

'3.10.10 | packaged by conda-forge | (main, Mar 24 2023, 20:08:06) [GCC 11.3.0]'

In [2]:
# !mamba create -n py311 -y
# !source /opt/conda/bin/activate py311 && mamba install python=3.11 jupyter mamba -y

# !sudo rm /opt/conda/bin/python3
# !sudo ln -sf /opt/conda/envs/py311/bin/python3 /opt/conda/bin/python3
# !sudo rm /opt/conda/bin/python3.7
# !sudo ln -sf /opt/conda/envs/py311/bin/python3 /opt/conda/bin/python3.7
# !sudo rm /opt/conda/bin/python
# !sudo ln -sf /opt/conda/envs/py311/bin/python3 /opt/conda/bin/python

In [3]:
!pip install optuna xgboost torch

In [4]:
# Utils imports
import pandas as pd
import numpy as np
import os
import joblib
import random

#Optimization imports
import optuna as opt

#Evaluation imports
from sklearn.metrics import accuracy_score, mean_absolute_error, mean_squared_error, r2_score

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [5]:
# Utils imports
import pandas as pd
import numpy as np
import os
import joblib
import random
import logging

#Optimization imports
import optuna as opt

#Evaluation imports
from sklearn.metrics import accuracy_score, mean_absolute_error, mean_squared_error, r2_score

In [6]:
import logging
#logging setup
from datetime import date


# Configure the logger
logger = logging.getLogger('Model Logger')
logger.setLevel(logging.DEBUG)

# Add a file handler to save log records to a file
log_file = f'{date.today().strftime("%Y_%m>%d")}.log'
file_handler = logging.FileHandler(log_file)
file_handler.setLevel(logging.DEBUG)

# Create a formatter to specify the format of the log records
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
file_handler.setFormatter(formatter)

# Add the file handler to the logger
logger.addHandler(file_handler)

# Log a message
logger.info('Logging Started')

In [7]:
#Model imports
from xgboost import XGBRegressor
import xgboost as xgb
# from pytorch_tabnet.tab_model import TabNetRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR

In [9]:
# Preparing data

df = pd.read_csv("/kaggle/input/fish-pred-data/smogn_syn_data.csv")
tar_col = "LC50 [-LOG(mol/L)]"
Syn = "isSyn"
isSyn = df[Syn]
X = df.drop([tar_col,Syn], axis=1)
Y = df[tar_col]


# PARAMS 
NUM_FOLDS = 15
NUM_TRIALS = 20
FOLD_DICT = joblib.load("/kaggle/input/fish-pred-data/train_test_fold_data.z")

In [10]:
# def train_main(X,Y,FOLD_DICT,model_name):
#     fold_data = {}
#     trial_fold = random.choice(range(NUM_FOLDS))
#     trial_train_index = FOLD_DICT[trial_fold]["train"]
#     trial_test_index = FOLD_DICT[trial_fold]["test"]
#     trial_X_main = X.iloc[trial_train_index, :].to_numpy(dtype=np.float64)
#     trial_Y_main = Y[trial_train_index].to_numpy(dtype=np.float64)
#     trial_X_test = X.iloc[trial_test_index, :].to_numpy(dtype=np.float64)
#     trial_Y_test = Y[trial_test_index].to_numpy(dtype=np.float64)
#     logger.info(f"[{model_name}] Hyper-Optim Started on fold {trial_fold}")
#     def objective(trial):
#         reg = XGBRegressor(
#                     n_estimators = trial.suggest_categorical("xgb_est",[4500,5000]),
#                     learning_rate = trial.suggest_categorical("xgb_lr",[0.01,3e-4,0.1]),
#                     booster = "dart",
#                     tree_method =  "gpu_hist",
#                     predictor = "gpu_predictor")
#         reg.fit(trial_X_main, trial_Y_main,
#                     eval_set = [(trial_X_test,trial_Y_test)],
#                     eval_metric = ["rmse"])
#         trial_Y_pred = reg.predict(trial_X_test)
#         error = mean_squared_error(trial_Y_pred, trial_Y_test, squared=False)
#         return error
#     study = opt.create_study(direction='minimize')
#     study.optimize(objective, n_trials = NUM_TRIALS)
#     logger.info(f"[{model_name}] Hyper-Optim Ended on fold {trial_fold}")
#     best_params = study.best_params
#     trial_data = study.get_trials()
#     logger.info(f"[{model_name}] Cross Validation loop started")
#     for fold in range(NUM_FOLDS):
#         logger.info(f"[{model_name}] Cross Validation loop started for fold {fold}")
#         out_data = {}
#         train_index = FOLD_DICT[fold]["train"]
#         test_index = FOLD_DICT[fold]["test"]
#         X_main = X.iloc[train_index, :].to_numpy(dtype=np.float64)
#         Y_main = Y[train_index].to_numpy(dtype=np.float64)
#         X_test = X.iloc[test_index, :].to_numpy(dtype=np.float64)
#         Y_test = Y[test_index].to_numpy(dtype=np.float64)
#         reg_main= XGBRegressor(**best_params)
#         booster = reg_main.get_booster()
#         reg_main.fit(X_main, Y_main,eval_set = [(X_test, Y_test)],eval_metric = ["rmse"])
#         Y_pred_main = reg_main.predict(X_test)
#         booster.save_model(f"model_{fold}.bin")
#         error_metrics_all = {
#             "mse_error" : mean_squared_error(Y_pred_main,Y_test),
#             "mae_error" : mean_absolute_error(Y_pred_main,Y_test),
#             "rmse_error" : mean_squared_error(Y_pred_main,Y_test, squared=False),
#             "r2_score" : r2_score(Y_pred_main,Y_test)}

#         out_data[model_name] = { "best_params"  : best_params,
#                                     "trial_data" : trial_data,
#                                 "error_metric_all": error_metrics_all,
#                                 "model_bin" : booster }
#         logger.info(f"[{model_name}] Cross Validation loop ended for fold {fold}")
#         fold_data[fold] = out_data
#     logger.info(f"[{model_name}] Main Training Loop Ended")
#     return fold_data
# model_data = train_main(X,Y,FOLD_DICT,model_name="knReg")

# other model sample codes

# def train_main(X,Y,FOLD_DICT,model_name):
#     fold_data = {}
#     trial_fold = random.choice(range(NUM_FOLDS))
#     trial_train_index = FOLD_DICT[trial_fold]["train"]
#     trial_test_index = FOLD_DICT[trial_fold]["test"]
#     trial_X_main = X.iloc[trial_train_index, :].to_numpy(dtype=np.float64)
#     trial_Y_main = Y[trial_train_index].to_numpy(dtype=np.float64)
#     trial_X_test = X.iloc[trial_test_index, :].to_numpy(dtype=np.float64)
#     trial_Y_test = Y[trial_test_index].to_numpy(dtype=np.float64)
#     logger.info(f"[{model_name}] Hyper-Optim Started on fold {trial_fold}")
#     def objective(trial):
#         reg = LogisticRegression(
#                     penalty = "l2")
#         reg.fit(trial_X_main, trial_Y_main)
#         trial_Y_pred = reg.predict(trial_X_test)
#         error = mean_squared_error(trial_Y_pred, trial_Y_test, squared=False)
#         return error
#     study = opt.create_study(direction='minimize')
#     study.optimize(objective, n_trials = NUM_TRIALS)
#     logger.info(f"[{model_name}] Hyper-Optim Ended on fold {trial_fold}")
#     best_params = study.best_params
#     trial_data = study.get_trials()
#     logger.info(f"[{model_name}] Cross Validation loop started")
#     for fold in range(NUM_FOLDS):
#         logger.info(f"[{model_name}] Cross Validation loop started for fold {fold}")
#         out_data = {}
#         train_index = FOLD_DICT[fold]["train"]
#         test_index = FOLD_DICT[fold]["test"]
#         X_main = X.iloc[train_index, :].to_numpy(dtype=np.float64)
#         Y_main = Y[train_index].to_numpy(dtype=np.float64)
#         X_test = X.iloc[test_index, :].to_numpy(dtype=np.float64)
#         Y_test = Y[test_index].to_numpy(dtype=np.float64)
#         reg_main= LogisticRegression(**best_params)
#         reg_main.fit(X_main, Y_main)
#         Y_pred_main = reg_main.predict(X_test)
#         joblib.dump(reg_main, f"{fold}_model.z")
#         error_metrics_all = {
#             "mse_error" : mean_squared_error(Y_pred_main,Y_test),
#             "mae_error" : mean_absolute_error(Y_pred_main,Y_test),
#             "rmse_error" : mean_squared_error(Y_pred_main,Y_test, squared=False),
#             "r2_score" : r2_score(Y_pred_main,Y_test)}

#         out_data[model_name] = { "best_params"  : best_params,
#                                     "trial_data" : trial_data,
#                                 "error_metric_all": error_metrics_all,
#                                 "model_bin" : reg_main }
#         logger.info(f"[{model_name}] Cross Validation loop ended for fold {fold}")
#         fold_data[fold] = out_data
#     logger.info(f"[{model_name}] Main Training Loop Ended")
#     return fold_data
# model_data = train_main(X,Y,FOLD_DICT,model_name="LogisticRegression")


# linear regression setup

def train_main(X,Y,FOLD_DICT,model_name):
    fold_data = {}
#     trial_fold = random.choice(range(NUM_FOLDS))
#     trial_train_index = FOLD_DICT[trial_fold]["train"]
#     trial_test_index = FOLD_DICT[trial_fold]["test"]
#     trial_X_main = X.iloc[trial_train_index, :].to_numpy(dtype=np.float64)
#     trial_Y_main = Y[trial_train_index].to_numpy(dtype=np.float64)
#     trial_X_test = X.iloc[trial_test_index, :].to_numpy(dtype=np.float64)
#     trial_Y_test = Y[trial_test_index].to_numpy(dtype=np.float64)
#     logger.info(f"[{model_name}] Hyper-Optim Started on fold {trial_fold}")
#     def objective(trial):
#         reg = LogisticRegression(
#                     penalty = "l2")
#         reg.fit(trial_X_main, trial_Y_main)
#         trial_Y_pred = reg.predict(trial_X_test)
#         error = mean_squared_error(trial_Y_pred, trial_Y_test, squared=False)
#         return error
#     study = opt.create_study(direction='minimize')
#     study.optimize(objective, n_trials = NUM_TRIALS)
#     logger.info(f"[{model_name}] Hyper-Optim Ended on fold {trial_fold}")
#     best_params = study.best_params
#     trial_data = study.get_trials()
#     logger.info(f"[{model_name}] Cross Validation loop started")
    for fold in range(NUM_FOLDS):
        logger.info(f"[{model_name}] Cross Validation loop started for fold {fold}")
        out_data = {}
        train_index = FOLD_DICT[fold]["train"]
        test_index = FOLD_DICT[fold]["test"]
        X_main = X.iloc[train_index, :].to_numpy(dtype=np.float64)
        Y_main = Y[train_index].to_numpy(dtype=np.float64)
        X_test = X.iloc[test_index, :].to_numpy(dtype=np.float64)
        Y_test = Y[test_index].to_numpy(dtype=np.float64)
        reg_main= LinearRegression()
        reg_main.fit(X_main, Y_main)
        Y_pred_main = reg_main.predict(X_test)
        joblib.dump(reg_main, f"{fold}_model.z")
        error_metrics_all = {
            "mse_error" : mean_squared_error(Y_pred_main,Y_test),
            "mae_error" : mean_absolute_error(Y_pred_main,Y_test),
            "rmse_error" : mean_squared_error(Y_pred_main,Y_test, squared=False),
            "r2_score" : r2_score(Y_pred_main,Y_test)}

        out_data[model_name] = {
                                "error_metric_all": error_metrics_all,
                                "model_bin" : reg_main }
        logger.info(f"[{model_name}] Cross Validation loop ended for fold {fold}")
        fold_data[fold] = out_data
    logger.info(f"[{model_name}] Main Training Loop Ended")
    return fold_data
model_data = train_main(X,Y,FOLD_DICT,model_name="LinearRegression")

In [11]:
joblib.dump(model_data,"model_data.z")
# Close the file handler
file_handler.close()